## Setup

In [1]:
!pip install -r requirements.txt

In [2]:
import numpy as np
import os
import pandas as pd
from rich import inspect

from utils import *

console = setup_rich()

In [3]:
filenames = ["discord-4nkq7191ray", "discord-fv8dqxpzsem"]
data_dir = ".data/csv/"
filenames = [os.path.join(data_dir, f"{f}.csv") for f in filenames]
train_data = pd.read_csv(filenames[1])
test_data = pd.read_csv(filenames[0])
train_data.head()

,text
0,Hehe
1,"20220127_210848.jpg"",""prependIcon"":""attachment..."
2,I think I'm gonna stream playing im excited
3,Time to impulse buy
4,Wait is pokemon diamond out?


## Pipeline demo

In [4]:
# Load model directly
import warnings

import torch
from transformers import *

warnings.filterwarnings("ignore")


model_name = "microsoft/Phi-3-mini-4k-instruct"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device="cpu",
)
pipe

c:\Users\nicol\Anaconda3\envs\llm-dev\Lib\site-packages\transformers\deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
loading file tokenizer.model from cache at C:\Users\nicol\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapshots\c1358f8a35e6d2af81890deffbbfa575b978c62f\tokenizer.model
loading file added_tokens.json from cache at C:\Users\nicol\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapshots\c1358f8a35e6d2af81890deffbbfa575b978c62f\added_tokens.json
loading file special_tokens_map.json from cache at C:\Users\nicol\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapshots\c1358f8a35e6d2af81890deffbbfa575b978c62f\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\nicol\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapsh

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Phi3ForCausalLM.

All the weights of Phi3ForCausalLM were initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at C:\Users\nicol\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapshots\c1358f8a35e6d2af81890deffbbfa575b978c62f\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}



<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x0000026F512DD810>

In [5]:
# pipe("Can you explain LangChain?")

In [6]:
inspect(pipe)

╭───────── <transformers.pipelines.text_generation.TextGenerationPipeline object at 0x0000026F512DD810> ──────────╮
│ def (text_inputs, **kwargs):                                                                                    │
│                                                                                                                 │
│ Language generation pipeline using any `ModelWithLMHead`. This pipeline predicts the words that will follow a   │
│ specified text prompt. When the underlying model is a conversational model, it can also accept one or more      │
│ chats,                                                                                                          │
│ in which case the pipeline will operate in chat mode and will continue the chat(s) by adding its response(s).   │
│ Each chat takes the form of a list of dicts, where each dict contains "role" and "content" keys.                │
│                                                                                                                 │
│       binary_output = False                                                                                     │
│          call_count = 0                                                                                         │
│ default_input_names = None                                                                                      │
│              device = device(type='cpu')                                                                        │
│   feature_extractor = None                                                                                      │
│           framework = 'pt'                                                                                      │
│     image_processor = None                                                                                      │
│           modelcard = None                                                                                      │
│                task = 'text-generation'                                                                         │
│         torch_dtype = torch.float32                                                                             │
│           XL_PREFIX = "\n    In 1991, the remains of Russian Tsar Nicholas II and his family (except for Alexei │
│                       and Maria) are discovered. The\n    voice of Nicholas's young son, Tsarevich Alexei       │
│                       Nikolaevich, narrates the remainder of the story. 1883 Western\n    Siberia, a young      │
│                       Grigori Rasputin is asked by his father and a group of men to perform magic. Rasputin has │
│                       a vision\n    and denounces one of the men as a horse thief. Although his father          │
│                       initially slaps him for making such an\n    accusation, Rasputin watches as the man is    │
│                       chased outside and beaten. Twenty years later, Rasputin sees a vision of\n    the Virgin  │
│                       Mary, prompting him to become a priest. Rasputin quickly becomes famous, with people,     │
│                       even a bishop,\n    begging for his blessing. <eod> </s> <eos>\n    "                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [7]:
sample = train_data.sample(1)["text"].values[0]
print(sample)


GOODY TWO SHOES


In [8]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
train_dataset

Dataset({
    features: ['text'],
    num_rows: 51305
})

In [9]:
sample = train_data.sample(1)["text"].values[0]
console.print(sample)
input_ids = tokenizer.encode(sample, return_tensors="pt")
input_ids

:WackyDebaters1:

tensor([[  584, 29956,   547, 29891, 10251, 10412, 29896, 29901]])

In [10]:
texts = train_data["text"].tolist()

In [11]:
texts_encodings = tokenizer(texts[:5], truncation=True, padding=True)

In [12]:
texts_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [13]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/51305 [00:00<?, ? examples/s]

Map:   0%|          | 0/4245 [00:00<?, ? examples/s]

In [14]:
# inspect(train_dataset)

## Training

In [15]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=32064, bias=False)
)

In [16]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=4,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "qkv_proj",
    ],
)
lora_model = get_peft_model(model, lora_config).to(device)
lora_model.print_trainable_parameters()

trainable params: 6,291,456 || all params: 3,827,371,008 || trainable%: 0.1644


In [17]:
output_dir = ".data/outputs/"

training_args = TrainingArguments(
    eval_strategy="epoch",
    bf16=True,
    gradient_accumulation_steps=256,
    learning_rate=2e-5,
    output_dir=output_dir,
    push_to_hub=False,
    remove_unused_columns=True,
    report_to="none",
    weight_decay=0.01,
    auto_find_batch_size=True,
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

PyTorch: setting up devices
Using auto half precision backend
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 51,305
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 256
  Total optimization steps = 600
  Number of trainable parameters = 6,291,456


  0%|          | 0/600 [00:00<?, ?it/s]

You are not running the flash-attention implementation, expect numerical differences.


In [ ]:
import math

eval_results = trainer.evaluate()
console.print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")